In [32]:
from plots import loadModel

In [2]:
%matplotlib inline

import setGPU
import os, h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm, Normalize
import glob

setGPU: Setting GPU to: 0


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [26]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, ZeroPadding2D, AveragePooling2D
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

In [4]:
disc_weight = '/nfshome/vitoriabp/hgcal_wgan/trial23/weights/discriminator_weights_epoch_1975_bs_128_lr_0.00005_latent_100_ncritic_5_clip_0.01_shuffle.h5'

In [29]:
def build_critic():

    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(2, 2), padding='same', input_shape=(16, 16, 55)))

    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(rate=0.25))

    model.add(ZeroPadding2D(padding=(2, 2)))

    model.add(Conv2D(filters=8, kernel_size=(2, 2), padding='valid'))

    model.add(LeakyReLU())
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(rate=0.25))

    model.add(ZeroPadding2D(padding=(2, 2)))

    model.add(Conv2D(filters=8, kernel_size=(2, 2), padding='valid'))

    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(rate=0.25))

    model.add(ZeroPadding2D(padding=(1, 1)))

    model.add(Conv2D(filters=8, kernel_size=(2, 2), padding='valid'))

    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(rate=0.25))

    model.add(AveragePooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1))

    model.summary()

    img = Input(shape=(16, 16, 55))
    validity = model(img)

    return Model(img, validity)

In [30]:
disc_model = build_critic()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 16, 16, 32)        7072      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 19, 19, 8)         1032      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 19, 19, 8)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 19, 19, 8)         32        
__________

In [33]:
def saveModel(model, name="regression"):
    '''
    Saves model as json file..
    :parameter model: model to be saved.
    :parameter name: name of the model to be saved.
    :type name: str
    :return: saved model.
    '''
    model_name = name
    model.summary()
    #model.save_weights('%s.h5' % model_name, overwrite=True)
    model_json = model.to_json()
    with open("%s.json" % model_name, "w") as json_file:
        json_file.write(model_json)

In [35]:
saveModel(disc_model, name="discriminator_trial23")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 16, 55)        0         
_________________________________________________________________
sequential_10 (Sequential)   (None, 1)                 9697      
Total params: 9,697
Trainable params: 9,649
Non-trainable params: 48
_________________________________________________________________


In [36]:
d_model = loadModel('discriminator_trial23.json')

In [37]:
d_model.load_weights(disc_weight)

In [38]:
f = h5py.File('/bigdata/shared/HGCAL_data/new_multi_small/no_pu/ntuple_merged_159_no_pu.h5', 'r')

In [40]:
list(f.keys())

['X', 'y']

In [41]:
imgs = np.asarray(f['X'])

In [43]:
imgs = imgs.squeeze()

In [44]:
imgs.shape

(58, 16, 16, 55)

In [46]:
pred = d_model.predict(imgs)

In [49]:
pred

array([[7.2917421e-05],
       [7.2203962e-05],
       [7.2728901e-05],
       [7.2584895e-05],
       [7.2700779e-05],
       [7.2648050e-05],
       [7.3133735e-05],
       [7.2077171e-05],
       [7.2723633e-05],
       [7.2571296e-05],
       [7.2829214e-05],
       [7.2271279e-05],
       [7.2904520e-05],
       [7.2389259e-05],
       [7.2889452e-05],
       [7.2101517e-05],
       [7.2667193e-05],
       [7.2145755e-05],
       [7.2869028e-05],
       [7.2518997e-05],
       [7.3268558e-05],
       [7.2239709e-05],
       [7.2907213e-05],
       [7.2128896e-05],
       [7.2629111e-05],
       [7.2359471e-05],
       [7.2572118e-05],
       [7.2352239e-05],
       [7.2693190e-05],
       [7.2685187e-05],
       [7.2828901e-05],
       [7.2629424e-05],
       [7.2854658e-05],
       [7.2415518e-05],
       [7.2464063e-05],
       [7.2160161e-05],
       [7.3105402e-05],
       [7.2338938e-05],
       [7.2738025e-05],
       [7.2597781e-05],
       [7.2218543e-05],
       [7.170407

In [50]:
gen_model1 = '/nfshome/vitoriabp/hgcal_wgan/trial23/weights/generator_model_bs_128_lr_0.00005_latent_100_ncritic_5_clip_0.01_shuffle.json'

In [51]:
all_g_weight1 = '/nfshome/vitoriabp/hgcal_wgan/trial23/weights/generator_weights_epoch_1975_bs_128_lr_0.00005_latent_100_ncritic_5_clip_0.01_shuffle.h5'

In [52]:
g = loadModel(gen_model1)

In [53]:
g.load_weights(all_g_weight1)

In [56]:
noise = np.random.normal(0, 1, (32, 100))
fake = g.predict(noise)

In [57]:
fake.shape

(32, 16, 16, 55)

In [58]:
pred_fake = d_model.predict(fake)

In [59]:
pred_fake

array([[7.2212017e-05],
       [7.1711438e-05],
       [7.2578005e-05],
       [7.2508585e-05],
       [7.2527255e-05],
       [7.2474082e-05],
       [7.2072078e-05],
       [7.2518298e-05],
       [7.2438939e-05],
       [7.2530500e-05],
       [7.2575458e-05],
       [7.2246352e-05],
       [7.2481329e-05],
       [7.2386647e-05],
       [7.2014431e-05],
       [7.2564326e-05],
       [7.2609793e-05],
       [7.2091658e-05],
       [7.2621071e-05],
       [7.2430266e-05],
       [7.2435854e-05],
       [7.2056013e-05],
       [7.2249299e-05],
       [7.1693088e-05],
       [7.2554714e-05],
       [7.2366041e-05],
       [7.2379829e-05],
       [7.1581300e-05],
       [7.2472787e-05],
       [7.1842478e-05],
       [7.2522875e-05],
       [7.2171642e-05]], dtype=float32)